In [37]:
def get_stock_history_eod(symbol="AAPL", from_yyyy_mm_dd = "all", to_yyyy_mm_dd = "all", api_token = config['api_key']):
 
    if from_yyyy_mm_dd != "all":
        url = "https://eodhistoricaldata.com/api/eod/" + symbol + ".US?from="+from_yyyy_mm_dd+"&to="+to_yyyy_mm_dd+"&api_token=" + config['api_key'] + "&period=d"
    else:
        url = "https://eodhistoricaldata.com/api/eod/" + symbol + ".US?&api_token=" + config['api_key'] + "&period=d"

    r = hfs.requests_retry_session().get(url)
    c = 0
    while r.status_code != requests.codes.ok:

        if r.status_code == 429: 
            #print("waiting 5 sec for ", symbol)
            if c == 5:
                print("c == 5 breaking", symbol)
                break
            time.sleep(random.randint(3, 6))
            try:
                r = hfs.requests_retry_session().get(url)
                c+=1
            except:
                continue

        else:
            print(symbol, " breaking : " + str(r.status_code) + " <> status code") 
            break
            #return pd.DataFrame([{'Ticker' : symbol,  'Open' : r.reason,  'Date' : r.status_code,  'Close': url}]).reset_index(drop=True)
            print("******* raising Exception *****",type(r.status_code), r.status_code)
            raise Exception(symbol, r.status_code, r.reason, url) 

    else:

        df = pd.read_csv(StringIO(r.text), skipfooter=0, engine='python').reset_index(drop=True) # 
        df['Ticker'] = symbol
        return df.reset_index(drop=True) 

In [38]:
ana_recs = pd.read_csv(f"./past_preds/splt_dt_{SPLIT_DATE}_result_ana.csv")
ana_recs = ana_recs[-pd.isnull(ana_recs['rec_tickers'])].reset_index(drop=True)
if 'Unnamed: 0' in ana_recs.columns:
    del ana_recs['Unnamed: 0']
print(ana_recs.shape)
ana_recs.head()

(23, 2)


,type,rec_tickers
0,industry_1_Consumer Defensive,"CLEU, SOVO, BSFC, MF, AGRI"
1,industry_2_Basic Materials,"CX, SVFD, MP, PRM, KGC"
2,industry_3_Technology,"GEN, FSLY, DTSS, AAOI, ACMR"
3,entry_in_EP_SC,"ACHR, AMSC, DWAC, ETRN, GDRX, HCTI, ICAD, IONQ, PACW, PLM, PLTR, SOFI, XPEV"
4,entry_in_EP_LC,"BBIO, CVNA, RIVN"


In [39]:
ana_rec_list = list(set([item for sublist in [x.split(",") for x in ana_recs['rec_tickers'].values] for item in sublist]))
len(ana_rec_list)

140

In [40]:
tickers = ana_rec_list
print("total #tickers: ", len(tickers))

args = ((symbol,      
        str(FROM_DATE).split()[0],   # From date
         str(date.today()), # to date
        config['api_key']) for symbol in tickers )

stocks_daily_ohlcv = hfs.get_data_from_API(tickers = tickers,
                                          func_to_run = get_stock_history_eod,
                                          func_args = args ) 

print("unique #tickers: ", stocks_daily_ohlcv['Ticker'].nunique()) 
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.tail(2) 

total #tickers:  140
chunk  1 :  140
total time:  1.9601819515228271
returning df
unique #tickers:  140
(120455, 8)


,Date,Open,High,Low,Close,Adjusted_close,Volume,Ticker
120453,2023-08-07,60.50,60.5,51.61,51.97,51.97,14754300,UPST
120454,2023-08-08,48.23,53.6,46.30,51.75,51.75,15968404,UPST


In [41]:
stocks_daily_ohlcv[stocks_daily_ohlcv['Ticker']=='SYSX'].tail(2)

,Date,Open,High,Low,Close,Adjusted_close,Volume,Ticker


In [42]:
stocks_daily_ohlcv['Date'] = pd.to_datetime(stocks_daily_ohlcv['Date']) 

In [43]:
print("SPLIT_DATE: ", SPLIT_DATE)
# Adjust for Splits 

stocks_daily_ohlcv['close_to_adj_close_ratio'] = (stocks_daily_ohlcv['Close'] / stocks_daily_ohlcv['Adjusted_close']) #.astype(int) 

stocks_daily_ohlcv['Adjusted_Open'] = np.where(stocks_daily_ohlcv['close_to_adj_close_ratio'] == 1, 
                                          stocks_daily_ohlcv['Open'],
                                          stocks_daily_ohlcv['Open'] / stocks_daily_ohlcv['close_to_adj_close_ratio'])

stocks_daily_ohlcv['Adjusted_High'] = np.where(stocks_daily_ohlcv['close_to_adj_close_ratio'] == 1, 
                                          stocks_daily_ohlcv['High'],
                                          stocks_daily_ohlcv['High'] / stocks_daily_ohlcv['close_to_adj_close_ratio'])

stocks_daily_ohlcv['Adjusted_Low'] = np.where(stocks_daily_ohlcv['close_to_adj_close_ratio'] == 1, 
                                          stocks_daily_ohlcv['Low'],
                                          stocks_daily_ohlcv['Low'] / stocks_daily_ohlcv['close_to_adj_close_ratio'])

stocks_daily_ohlcv['Adjusted_Volume'] = np.where(stocks_daily_ohlcv['close_to_adj_close_ratio'] == 1, 
                                          stocks_daily_ohlcv['Volume'],
                                          stocks_daily_ohlcv['Volume'] * stocks_daily_ohlcv['close_to_adj_close_ratio'])

stocks_daily_ohlcv = stocks_daily_ohlcv[['Ticker', 'Date', 'Adjusted_Open', 'Adjusted_High', 'Adjusted_Low', 
                                         'Adjusted_close', 'Adjusted_Volume', 'close_to_adj_close_ratio']] 
print(stocks_daily_ohlcv.shape)
stocks_daily_ohlcv.tail(2) 

SPLIT_DATE:  2023-08-07
(120455, 8)


,Ticker,Date,Adjusted_Open,Adjusted_High,Adjusted_Low,Adjusted_close,Adjusted_Volume,close_to_adj_close_ratio
120453,UPST,2023-08-07,60.50,60.5,51.61,51.97,14754300.0,1.0
120454,UPST,2023-08-08,48.23,53.6,46.30,51.75,15968404.0,1.0


In [44]:
stocks_daily_ohlcv['Actual_tomo_High'] = stocks_daily_ohlcv.groupby('Ticker')['Adjusted_High'].shift(-1)
stocks_daily_ohlcv['Actual_tomo_Open'] = stocks_daily_ohlcv.groupby('Ticker')['Adjusted_Open'].shift(-1)
stocks_daily_ohlcv['Actual_tomo_Low'] = stocks_daily_ohlcv.groupby('Ticker')['Adjusted_Low'].shift(-1)
stocks_daily_ohlcv['Actual_tomo_Close'] = stocks_daily_ohlcv.groupby('Ticker')['Adjusted_close'].shift(-1)

stocks_daily_ohlcv['Actual_High_hike%'] = 100 * (stocks_daily_ohlcv['Actual_tomo_High'] - stocks_daily_ohlcv['Adjusted_close']) / stocks_daily_ohlcv['Adjusted_close']
stocks_daily_ohlcv['Actual_High_frmOpen_hike%'] = 100 * (stocks_daily_ohlcv['Actual_tomo_High'] - stocks_daily_ohlcv['Actual_tomo_Open']) / stocks_daily_ohlcv['Actual_tomo_Open']

stocks_daily_ohlcv['Actual_Low_hike%'] = 100 * (stocks_daily_ohlcv['Actual_tomo_Low'] - stocks_daily_ohlcv['Adjusted_close']) / stocks_daily_ohlcv['Adjusted_close']
stocks_daily_ohlcv['Actual_Low_frmOpen_hike%'] = 100 * (stocks_daily_ohlcv['Actual_tomo_Low'] - stocks_daily_ohlcv['Actual_tomo_Open']) / stocks_daily_ohlcv['Actual_tomo_Open']
 
stocks_daily_ohlcv.tail(2) 

,Ticker,Date,Adjusted_Open,Adjusted_High,Adjusted_Low,Adjusted_close,Adjusted_Volume,close_to_adj_close_ratio,Actual_tomo_High,Actual_tomo_Open,Actual_tomo_Low,Actual_tomo_Close,Actual_High_hike%,Actual_High_frmOpen_hike%,Actual_Low_hike%,Actual_Low_frmOpen_hike%
120453,UPST,2023-08-07,60.50,60.5,51.61,51.97,14754300.0,1.0,53.6,48.23,46.3,51.75,3.136425,11.134149,-10.91014,-4.001659
120454,UPST,2023-08-08,48.23,53.6,46.30,51.75,15968404.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
stocks_daily_ohlcv = stocks_daily_ohlcv[stocks_daily_ohlcv['Date']==SPLIT_DATE].reset_index(drop=True) 
stocks_daily_ohlcv['Ticker'] = stocks_daily_ohlcv['Ticker'].apply(lambda x: x.strip()) 
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.head(2) 
 

(140, 16)


,Ticker,Date,Adjusted_Open,Adjusted_High,Adjusted_Low,Adjusted_close,Adjusted_Volume,close_to_adj_close_ratio,Actual_tomo_High,Actual_tomo_Open,Actual_tomo_Low,Actual_tomo_Close,Actual_High_hike%,Actual_High_frmOpen_hike%,Actual_Low_hike%,Actual_Low_frmOpen_hike%
0,PATH,2023-08-07,16.11,16.27,15.760,16.19,5340680.0,1.0,15.92,15.81,15.40,15.83,-1.667696,0.695762,-4.879555,-2.593295
1,ATRA,2023-08-07,2.41,2.43,2.235,2.25,1593800.0,1.0,2.36,2.24,2.14,2.27,4.888889,5.357143,-4.888889,-4.464286


In [46]:
def get_numbers(func, calcs_df, ticker_list): 
    this_cat_df = calcs_df[calcs_df['Ticker'].isin(ticker_list)].reset_index(drop=True)
    if func == 'min':
        return this_cat_df[['Actual_High_frmOpen_hike%', 'Actual_Low_frmOpen_hike%']].min().values
    if func == 'max':
        return this_cat_df[['Actual_High_frmOpen_hike%','Actual_Low_frmOpen_hike%']].max().values
    if func == 'mean':
        return this_cat_df[['Actual_High_frmOpen_hike%','Actual_Low_frmOpen_hike%']].mean().values

ana_recs[['High_frmOpen_hike%_Mean','Low_frmOpen_hike%_Mean']] = ana_recs.apply(lambda x: get_numbers('mean', stocks_daily_ohlcv, [y.strip() for y in x['rec_tickers'].split(',')]), axis=1, result_type="expand" )
ana_recs[['High_frmOpen_hike%_Min','Low_frmOpen_hike%_Min']] = ana_recs.apply(lambda x: get_numbers('min', stocks_daily_ohlcv, [y.strip() for y in x['rec_tickers'].split(',')]), axis=1, result_type="expand" )
ana_recs[['High_frmOpen_hike%_Max','Low_frmOpen_hike%_Max']] = ana_recs.apply(lambda x: get_numbers('max', stocks_daily_ohlcv, [y.strip() for y in x['rec_tickers'].split(',')]), axis=1, result_type="expand" )
ana_recs.round(2) 
 

,type,rec_tickers,High_frmOpen_hike%_Mean,Low_frmOpen_hike%_Mean,High_frmOpen_hike%_Min,Low_frmOpen_hike%_Min,High_frmOpen_hike%_Max,Low_frmOpen_hike%_Max
0,industry_1_Consumer Defensive,"CLEU, SOVO, BSFC, MF, AGRI",3.32,-5.30,0.13,-15.89,10.70,-0.51
1,industry_2_Basic Materials,"CX, SVFD, MP, PRM, KGC",6.56,-1.04,2.12,-1.59,20.37,-0.41
2,industry_3_Technology,"GEN, FSLY, DTSS, AAOI, ACMR",4.12,-4.74,0.22,-14.47,5.25,-2.48
3,entry_in_EP_SC,"ACHR, AMSC, DWAC, ETRN, GDRX, HCTI, ICAD, IONQ, PACW, PLM, PLTR, SOFI, XPEV",3.25,-2.42,0.00,-7.95,12.60,0.00
4,entry_in_EP_LC,"BBIO, CVNA, RIVN",4.31,-1.27,0.93,-3.34,6.18,-0.49
5,consolidating_5d,"ACHR, AMC, AVTR, ENVX, EOSE, FUBO, GDRX, INVZ, IQ, MARA, MRSN, SNAP, SWN",3.08,-2.90,0.00,-8.61,5.98,-0.47
6,extended_fall_days,"ARAV, CLVT, QS",2.27,-4.15,0.04,-7.25,6.39,-2.32
7,extended_rise_days,"AHI, MF, RAD, MGIH",15.18,-7.66,8.21,-12.09,24.09,-5.04
8,ramp_2_days_for_dpb_or_frd,"AAOI, DTSS, MF, SVFD, TDS",11.50,-4.60,4.90,-14.47,22.86,0.00
9,FGD,"NKTR, SVFD, BSFC, DTSS, MGIH",9.63,-11.37,0.93,-15.89,20.37,-0.93
